In [ ]:
from transformers import AutoTokenizer
from deepseek_v3.modeling_deepseek import DeepseekV3ForCausalLM
import shutil
import torch
import os
from dotenv import load_dotenv

load_dotenv()

model_name="DeepSeek-V3-4@1-unhealed-v0.1"

shutil.copy('UNHEALED_README.md', os.path.join(model_name, "README.md"))

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = DeepseekV3ForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cpu", ## This should distribute automatically on all cpu
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.push_to_hub(model_name.replace('@','a'))
tokenizer.push_to_hub(model_name.replace('@','a'))

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
DeepseekV3ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]